## Dependencies:

In [2]:
import pandas as pd
import numpy as np

from src.data.utils import read_data

## Configuration

In [3]:
pd.set_option('max_colwidth', 500)
pd.set_option('display.max_rows', 500)

## Read data

In [5]:
data_path = '../data/raw/FoodData_Central_csv_2023-04-20/'

food = read_data(f'{data_path}food.csv')
survey_fndds_food = read_data(f'{data_path}survey_fndds_food.csv')
nutrient_data = read_data(f'{data_path}food_nutrient.csv')
nutrient_info = read_data(f'{data_path}nutrient.csv')
input_food = read_data(f'{data_path}input_food.csv')
fndds_ingredient_nutrient_value = read_data(f'{data_path}fndds_ingredient_nutrient_value.csv')
fndds_derivation = read_data(f'{data_path}fndds_derivation.csv')
#category = pd.read_csv(f'{data_path}food_category.csv')
component = read_data(f'{data_path}food_component.csv')
category = read_data(f'{data_path}wweia_food_category.csv')

/mnt/c/Users/dhdzm/Documents/projects/ingeineria_en_alimentos/ingenieria_en_alimentos/ai_in_food/src/data/utils.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [7]:
nutrient_info.loc[
    nutrient_info.nutrient_nbr.notna(), 'nutrient_nbr_int'
] = nutrient_info.loc[nutrient_info.nutrient_nbr.notna()].nutrient_nbr.astype('int')

fndss_dt = food.data_type == 'survey_fndds_food'
relevant_list = ['description', 'fdc_id', 'food_category_id', 'publication_date']
food_survey_food = survey_fndds_food.merge(food[fndss_dt][relevant_list], on='fdc_id', how='left')

food_survey_food = food_survey_food\
.merge(category, how='left', left_on='wweia_category_code', right_on='wweia_food_category')\
.drop(['wweia_category_code', 'food_category_id', 'start_date', 'end_date', 'publication_date'], axis=1)

#del food
#del survey_fndds_food
#del category

In [8]:
def get_food_information(test_food):
    food_survey_food[food_survey_food.fdc_id == test_food]

    ingredients = input_food[input_food.fdc_id == test_food].sort_values('seq_num')
    codes = ingredients.sr_code.unique()

    nutrient_info_reduced = nutrient_info[nutrient_info.nutrient_nbr_int.notna()][['name', 'unit_name', 'nutrient_nbr_int']]
    specific_nutrient_val = fndds_ingredient_nutrient_value\
    [fndds_ingredient_nutrient_value['ingredient code'].isin(codes)]\
    .drop_duplicates(['ingredient code', 'Nutrient code'])\
    .merge(
        nutrient_info_reduced,
        left_on='Nutrient code',
        right_on='nutrient_nbr_int')\
    [['ingredient code', 'Nutrient code', 'Nutrient value', 'name', 'unit_name']]
    specific_nutrient_val = specific_nutrient_val[specific_nutrient_val['Nutrient value']!=0]\
    .sort_values(['Nutrient code', 'Nutrient value']).reset_index(drop=True)\
    .drop_duplicates(['ingredient code', 'Nutrient code'])

    missing_codes = list(set(codes) - set(specific_nutrient_val['ingredient code'].unique()))

    missing_ingredients = ingredients[ingredients.sr_code.isin(missing_codes)]

    nutrient_info_reduced = nutrient_info[nutrient_info.nutrient_nbr_int.notna()][['name', 'unit_name', 'nutrient_nbr_int']]
    specific_nutrient_val = fndds_ingredient_nutrient_value\
    [fndds_ingredient_nutrient_value['ingredient code'].isin(codes)]\
    .drop_duplicates(['ingredient code', 'Nutrient code'])\
    .merge(
        nutrient_info_reduced,
        left_on='Nutrient code',
        right_on='nutrient_nbr_int')\
    [['ingredient code', 'Nutrient code', 'Nutrient value', 'name', 'unit_name']]
    specific_nutrient_val = specific_nutrient_val[specific_nutrient_val['Nutrient value']!=0]\
    .sort_values(['ingredient code', 'Nutrient code', 'Nutrient value'])\
    .reset_index(drop=True)\
    .drop_duplicates(['ingredient code', 'Nutrient code'])
    
    ingredients.index = ingredients.index.map(str)
    missing_ingredients.index = missing_ingredients.index.map(str)
    specific_nutrient_val.index = specific_nutrient_val.index.map(str)
    
    return ingredients, missing_ingredients, specific_nutrient_val

In [9]:
food_survey_food_resume = food_survey_food.groupby('wweia_food_category').agg(
    count_vals=('food_code',  'count'),
    categ=('wweia_food_category_description',  'first'),
    description_text=('description', lambda x: x.to_list()),
    fdc_ids = ('fdc_id', lambda x: x.to_list())
)

In [ ]:
#food_survey_food_resume.to_csv('../data/interim/survey_food_resume.csv')

### We can select specific food categories and see which products are there.

In [12]:
milk = 1002, 1004, 1006, 1008, 1404, 9602
cheese = 1602, 1604
yogurt = 1820, 1822, 9010, 
meat = 2002, 2004, 2006, 2008, 2010, 2602, 2604, 9007
birds = 2202, 2206
fish = 2402
rice = 4002
pasta = 4004
bread = 4202
juice = 7002, 7004, 7006, 7204, 9202
tea = 7304
beer = 7502
wine = 7504
water = 7702, 7704, 9204
cream = 8008
mayonnaise = 8010
sugar = 8802, 8804

### Now we can select the desired products in each category, and by manual labor delete the ones that might not work. 

In [13]:
food_dataframe = {}

In [14]:
food_survey_food[food_survey_food.wweia_food_category.isin(milk)]

,fdc_id,food_code,description,wweia_food_category,wweia_food_category_description
0,2340760,11000000,"Milk, human",9602,Human milk
1,2340761,11100000,"Milk, NFS",1004,"Milk, reduced fat"
2,2340762,11111000,"Milk, whole",1002,"Milk, whole"
3,2340763,11112110,"Milk, reduced fat (2%)",1004,"Milk, reduced fat"
4,2340764,11112210,"Milk, low fat (1%)",1006,"Milk, lowfat"
5,2340765,11113000,"Milk, fat free (skim)",1008,"Milk, nonfat"
6,2340766,11114300,"Milk, lactose free, low fat (1%)",1006,"Milk, lowfat"
7,2340767,11114320,"Milk, lactose free, fat free (skim)",1008,"Milk, nonfat"
8,2340768,11114330,"Milk, lactose free, reduced fat (2%)",1004,"Milk, reduced fat"
9,2340769,11114350,"Milk, lactose free, whole",1002,"Milk, whole"


In [15]:
not_categ = 2340899, 2340886, 2340875, 2340869, 2340867, 2340861, 2340856, 2340787, 2340788, 2340790, 2340792, 2340892
isin_criteria = food_survey_food.wweia_food_category.isin(milk)
food_dataframe['milk'] = food_survey_food[isin_criteria].set_index('fdc_id').drop(list(not_categ)).drop('food_code', axis=1)

not_categ = 2341154, 2341155, 2341160, 2341182, 2341183, 2341183, 2341221
isin_criteria = food_survey_food.wweia_food_category.isin(cheese)
food_dataframe['cheese'] = food_survey_food[isin_criteria].set_index('fdc_id').drop(list(not_categ)).drop('food_code', axis=1)

not_categ = None
isin_criteria = food_survey_food.wweia_food_category.isin(yogurt)
food_dataframe['yogurt'] = food_survey_food[isin_criteria].set_index('fdc_id').drop('food_code', axis=1)

In [17]:
relevant_indexes = []
for element in food_dataframe.values():
    relevant_indexes += element.index.to_list()

In [45]:
test_nb = food_survey_food.sample(1).iloc[0].fdc_id
ingredients, _, _ = get_food_information(test_nb)

In [53]:
food_survey_food[food_survey_food.fdc_id==test_nb]

,fdc_id,food_code,description,wweia_food_category,wweia_food_category_description
4878,2345638,75603020,"Celery soup, cream of, prepared with water, home recipe, canned or ready-to-serve",3802,Soups


In [52]:
ingredients.sr_description

17663    Soup, cream of celery, canned, condensed
17664             Beverages, water, tap, drinking
Name: sr_description, dtype: object

### Here we get the ingredients and nutrition information of the ingredients. 

In [19]:
food_ingredient_nutrition_info = {}
for food in relevant_indexes:
    food_ingredient_nutrition_info[food] = {}
    ingredients, missing_ingredients, nutrition = get_food_information(food)
        
    food_ingredient_nutrition_info[food]['ingredients'] = ingredients[['sr_code', 'sr_description']].to_dict()
    food_ingredient_nutrition_info[food]['missing_ingredients'] = missing_ingredients.sr_code.to_dict()
    food_ingredient_nutrition_info[food]['nutrition'] = nutrition.to_dict()
    food_ingredient_nutrition_info[food]['unique_ingredients_code'] = ingredients.sr_code.unique()
    food_ingredient_nutrition_info[food]['unique_ingredients'] = ingredients.sr_description.unique()

In [20]:
food_ingredient_nutrition_info

{2340760: {'ingredients': {'sr_code': {'0': 1107},
   'sr_description': {'0': 'Milk, human, mature, fluid (For Reference Only)'}},
  'missing_ingredients': {},
  'nutrition': {'ingredient code': {'0': 1107,
    '1': 1107,
    '2': 1107,
    '4': 1107,
    '5': 1107,
    '6': 1107,
    '8': 1107,
    '9': 1107,
    '10': 1107,
    '11': 1107,
    '12': 1107,
    '13': 1107,
    '14': 1107,
    '15': 1107,
    '16': 1107,
    '17': 1107,
    '18': 1107,
    '19': 1107,
    '22': 1107,
    '24': 1107,
    '25': 1107,
    '26': 1107,
    '27': 1107,
    '28': 1107,
    '29': 1107,
    '30': 1107,
    '31': 1107,
    '32': 1107,
    '33': 1107,
    '34': 1107,
    '35': 1107,
    '36': 1107,
    '37': 1107,
    '38': 1107,
    '39': 1107,
    '40': 1107,
    '41': 1107,
    '42': 1107,
    '43': 1107,
    '44': 1107,
    '45': 1107,
    '46': 1107,
    '47': 1107,
    '48': 1107,
    '49': 1107,
    '50': 1107},
   'Nutrient code': {'0': 203,
    '1': 204,
    '2': 205,
    '4': 208,
    '5

In [14]:
food_ingredient_nutrition_info_processed = pd.DataFrame(food_ingredient_nutrition_info).T
food_ingredient_nutrition_info_processed.to_csv('../data/interim/food_ingredient_nutrition_info_processed.csv')

In [15]:
food_ingredient_nutrition_info_processed

,ingredients,missing_ingredients,nutrition,unique_ingredients_code,unique_ingredients
2340760,"{'sr_code': {'0': 1107}, 'sr_description': {'0': 'Milk, human, mature, fluid (For Reference Only)'}}",{},"{'ingredient code': {'0': 1107, '1': 1107, '2': 1107, '4': 1107, '5': 1107, '6': 1107, '8': 1107, '9': 1107, '10': 1107, '11': 1107, '12': 1107, '13': 1107, '14': 1107, '15': 1107, '16': 1107, '17': 1107, '18': 1107, '19': 1107, '22': 1107, '24': 1107, '25': 1107, '26': 1107, '27': 1107, '28': 1107, '29': 1107, '30': 1107, '31': 1107, '32': 1107, '33': 1107, '34': 1107, '35': 1107, '36': 1107, '37': 1107, '38': 1107, '39': 1107, '40': 1107, '41': 1107, '42': 1107, '43': 1107, '44': 1107, '45...",[1107],"[Milk, human, mature, fluid (For Reference Only)]"
2340761,"{'sr_code': {'1': 1077, '4': 1079, '2': 1082, '3': 1085}, 'sr_description': {'1': 'Milk, whole, 3.25% milkfat, with added vitamin D', '4': 'Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D', '2': 'Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D', '3': 'Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)'}}",{},"{'ingredient code': {'0': 1077, '1': 1077, '2': 1077, '4': 1077, '5': 1077, '6': 1077, '8': 1077, '9': 1077, '10': 1077, '11': 1077, '12': 1077, '13': 1077, '14': 1077, '15': 1077, '16': 1077, '17': 1077, '18': 1077, '21': 1077, '23': 1077, '24': 1077, '25': 1077, '26': 1077, '27': 1077, '28': 1077, '29': 1077, '30': 1077, '31': 1077, '32': 1077, '33': 1077, '34': 1077, '35': 1077, '36': 1077, '37': 1077, '38': 1077, '39': 1077, '40': 1077, '41': 1077, '42': 1077, '43': 1077, '44': 1077, '45...","[1077, 1079, 1082, 1085]","[Milk, whole, 3.25% milkfat, with added vitamin D, Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D, Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D, Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)]"
2340762,"{'sr_code': {'5': 1077}, 'sr_description': {'5': 'Milk, whole, 3.25% milkfat, with added vitamin D'}}",{},"{'ingredient code': {'0': 1077, '1': 1077, '2': 1077, '4': 1077, '5': 1077, '6': 1077, '8': 1077, '9': 1077, '10': 1077, '11': 1077, '12': 1077, '13': 1077, '14': 1077, '15': 1077, '16': 1077, '17': 1077, '18': 1077, '21': 1077, '23': 1077, '24': 1077, '25': 1077, '26': 1077, '27': 1077, '28': 1077, '29': 1077, '30': 1077, '31': 1077, '32': 1077, '33': 1077, '34': 1077, '35': 1077, '36': 1077, '37': 1077, '38': 1077, '39': 1077, '40': 1077, '41': 1077, '42': 1077, '43': 1077, '44': 1077, '45...",[1077],"[Milk, whole, 3.25% milkfat, with added vitamin D]"
2340763,"{'sr_code': {'6': 1079}, 'sr_description': {'6': 'Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D'}}",{},"{'ingredient code': {'0': 1079, '1': 1079, '2': 1079, '4': 1079, '5': 1079, '6': 1079, '8': 1079, '9': 1079, '10': 1079, '11': 1079, '12': 1079, '13': 1079, '14': 1079, '15': 1079, '16': 1079, '17': 1079, '18': 1079, '21': 1079, '23': 1079, '24': 1079, '25': 1079, '26': 1079, '27': 1079, '28': 1079, '29': 1079, '30': 1079, '31': 1079, '32': 1079, '33': 1079, '34': 1079, '35': 1079, '36': 1079, '37': 1079, '38': 1079, '39': 1079, '40': 1079, '41': 1079, '42': 1079, '43': 1079, '44': 1079, '45...",[1079],"[Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D]"
2340764,"{'sr_code': {'7': 1082}, 'sr_description': {'7': 'Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D'}}",{},"{'ingredient code': {'0': 1082, '1': 1082, '2': 1082, '4': 1082, '5': 1082, '6': 1082, '8': 1082, '9': 1082, '10': 1082, '11': 1082, '12': 1082, '13': 1082, '14': 1082, '15': 1082, '16': 1082, '17': 1082, '18': 1082, '21': 1082, '23': 1082, '24': 1082, '25': 1082, '26': 1082, '27': 1082, '28': 1082, '29': 1082, '30': 1082, '31': 1082, '32': 1082, '33': 1082, '34': 1082, '35': 1082, '36': 1082, '37': 1082, '38': 1082, '39': 1082, '40': 1082, '41': 1082, '42': 1082, '43': 1082, '44': 1082, '45...",[1082],"[Mil

### Similarity between specific products (if they belong to the same category, then it's a 1, if not, it's a 0)

In [16]:
response_dataframes = []
for key in food_dataframe.keys():
    for key2 in food_dataframe.keys():
        response=0
        if key==key2:
            response=1
        response_dataframe = food_dataframe[key].reset_index()[['fdc_id']]
        response_dataframe['fdc_id_to'] = [response_dataframe.fdc_id.to_list()]*len(response_dataframe)
        response_dataframe = response_dataframe.explode('fdc_id_to').reset_index(drop=True)
        response_dataframe['similar'] = response
        response_dataframes.append(response_dataframe)

In [17]:
response_dataframe = pd.concat(response_dataframes)
response_dataframe = response_dataframe.drop_duplicates()

In [19]:
response_dataframe

,fdc_id,fdc_id_to,similar
0,2340760,2340760,1
1,2340760,2340761,1
2,2340760,2340762,1
3,2340760,2340763,1
4,2340760,2340764,1
...,...,...,...
1084,2343045,2340823,1
1085,2343045,2340830,1
1086,2343045,2340844,1
1087,2343045,2340845,1


In [18]:
response_dataframe.to_csv('../data/interim/response_dataframe.csv')